In [16]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold, cross_val_score

In [17]:
df = pd.read_csv('data/spam.csv')[['category', 'email']]
X, y = df['email'].values, df['category']

In [18]:
X.shape, y.shape

((5572,), (5572,))

In [19]:
K = 100
count = CountVectorizer(analyzer='word')
word_counts = count.fit_transform(X).toarray()
X = SelectKBest(chi2, k=K).fit_transform(word_counts,y)
X = pd.DataFrame(X, columns=count.get_feature_names_out()[0:K])

In [20]:
X.shape

(5572, 100)

In [21]:
pd.DataFrame(X).describe()

,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,02,...,0870241182716,08702490080,08702840625,08704050406,08704439680,08704439680ts,08706091795,0870737910216yrs,08707500020,08707509020
count,5572.000000,5572.000000,5572.000000,5572.000000,5572.00000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,...,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000
mean,0.005205,0.007358,0.007358,0.004307,0.00323,0.012742,0.006102,0.009332,0.009153,0.004487,...,0.004307,0.006102,0.003769,0.022613,0.004307,0.013460,0.006999,0.017050,0.017947,0.122218
std,0.076788,0.085472,0.085472,0.068180,0.05675,0.112170,0.077883,0.096161,0.095241,0.066839,...,0.065494,0.084515,0.061281,0.167955,0.065494,0.124239,0.097286,0.129468,0.132770,0.385933
min,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,1.000000,1.000000,2.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,2.000000,1.000000,3.000000,1.000000,2.000000,2.000000,1.000000,1.000000,4.000000


In [22]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)
results = cross_val_score(GaussianNB(), X, y, cv=10, n_jobs=-1, scoring='accuracy')

In [23]:
'%.3f +/- %.3f' %(np.mean(results), np.std(results))

'0.967 +/- 0.005'